# Permuted Pixel MNIST Demo 
Light weighted demo of our DilatedRNN on Pixel MNist with permutation.

In [1]:
import sys
sys.path.append("./models")
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from classification_models import drnn_classification

In [2]:
# configurations
data_dir = "./MNIST_data"
n_steps = 28*28
input_dims = 1
n_classes = 10 

# model config
cell_type = "GRU"
assert(cell_type in ["RNN", "LSTM", "GRU"])
hidden_structs = [20, 30, 40, 50]
dilations = [4, 8, 16, 32]
assert(len(hidden_structs) == len(dilations))

# learning config
batch_size = 128
learning_rate = 1.0e-3
training_iters = batch_size * 30000
testing_step = 5000
display_step = 100

# permutation seed 
seed = 92916

In [3]:
mnist = input_data.read_data_sets(data_dir, one_hot=True)
if 'seed' in globals():
    rng_permute = np.random.RandomState(seed)
    idx_permute = rng_permute.permutation(n_steps)
else:
    idx_permute = np.random.permutation(n_steps)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# build computation graph
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, n_steps, input_dims])
y = tf.placeholder(tf.float32, [None, n_classes])    
global_step = tf.Variable(0, name='global_step', trainable=False)

# build prediction graph
print ("==> Building a dRNN with %s cells" %cell_type)
pred = drnn_classification(x, hidden_structs, dilations, n_steps, n_classes, input_dims, cell_type)

# build loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate, 0.9).minimize(cost, global_step=global_step)

# evaluation model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

==> Building a dRNN with GRU cells
Building layer: multi_dRNN_dilation_4, input length: 784, dilation rate: 4, input dim: 1.
=====> Input length for sub-RNN: 196
Building layer: multi_dRNN_dilation_8, input length: 784, dilation rate: 8, input dim: 20.
=====> Input length for sub-RNN: 98
Building layer: multi_dRNN_dilation_16, input length: 784, dilation rate: 16, input dim: 30.
=====> Input length for sub-RNN: 49
Building layer: multi_dRNN_dilation_32, input length: 784, dilation rate: 32, input dim: 40.
=====> 16 time points need to be padded. 
=====> Input length for sub-RNN: 25


In [5]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [6]:
step = 0
train_results = []
validation_results = []
test_results = []

while step * batch_size < training_iters:
    batch_x, batch_y = mnist.train.next_batch(batch_size)    
    batch_x = batch_x[:, idx_permute]
    batch_x = batch_x.reshape([batch_size, n_steps, input_dims])

    feed_dict = {
        x : batch_x,
        y : batch_y
    }
    cost_, accuracy_, step_,  _ = sess.run([cost, accuracy, global_step, optimizer], feed_dict=feed_dict)    
    train_results.append((step_, cost_, accuracy_))    

    if (step + 1) % display_step == 0:
        print ("Iter " + str(step + 1) + ", Minibatch Loss: " + "{:.6f}".format(cost_) \
        + ", Training Accuracy: " + "{:.6f}".format(accuracy_))
             
    if (step + 1) % testing_step == 0:
        
        # validation performance
        batch_x = mnist.validation.images
        batch_y = mnist.validation.labels

        # permute the data
        batch_x = batch_x[:, idx_permute]        
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy__, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        validation_results.append((step_, cost_, accuracy__))
        
        # test performance
        batch_x = mnist.test.images
        batch_y = mnist.test.labels
        batch_x = batch_x[:, idx_permute]        
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy_, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        test_results.append((step_, cost_, accuracy_))        
        print ("========> Validation Accuarcy: " + "{:.6f}".format(accuracy__) \
        + ", Testing Accuarcy: " + "{:.6f}".format(accuracy_)) 
    step += 1

Iter 100, Minibatch Loss: 1.967738, Training Accuracy: 0.273438
Iter 200, Minibatch Loss: 1.391636, Training Accuracy: 0.523438
Iter 300, Minibatch Loss: 1.121808, Training Accuracy: 0.593750
Iter 400, Minibatch Loss: 0.956666, Training Accuracy: 0.687500
Iter 500, Minibatch Loss: 0.772850, Training Accuracy: 0.765625
Iter 600, Minibatch Loss: 0.571083, Training Accuracy: 0.781250
Iter 700, Minibatch Loss: 0.716546, Training Accuracy: 0.750000
Iter 800, Minibatch Loss: 0.769885, Training Accuracy: 0.734375
Iter 900, Minibatch Loss: 0.695285, Training Accuracy: 0.789062
Iter 1000, Minibatch Loss: 0.643893, Training Accuracy: 0.773438
Iter 1100, Minibatch Loss: 0.609298, Training Accuracy: 0.789062
Iter 1200, Minibatch Loss: 0.355377, Training Accuracy: 0.867188
Iter 1300, Minibatch Loss: 0.485463, Training Accuracy: 0.843750
Iter 1400, Minibatch Loss: 0.464110, Training Accuracy: 0.867188
Iter 1500, Minibatch Loss: 0.388008, Training Accuracy: 0.890625
Iter 1600, Minibatch Loss: 0.48237

Iter 12500, Minibatch Loss: 0.071288, Training Accuracy: 0.984375
Iter 12600, Minibatch Loss: 0.064416, Training Accuracy: 0.976562
Iter 12700, Minibatch Loss: 0.065291, Training Accuracy: 0.976562
Iter 12800, Minibatch Loss: 0.080923, Training Accuracy: 0.968750
Iter 12900, Minibatch Loss: 0.083907, Training Accuracy: 0.984375
Iter 13000, Minibatch Loss: 0.040036, Training Accuracy: 0.984375
Iter 13100, Minibatch Loss: 0.102277, Training Accuracy: 0.984375
Iter 13200, Minibatch Loss: 0.088387, Training Accuracy: 0.968750
Iter 13300, Minibatch Loss: 0.155000, Training Accuracy: 0.953125
Iter 13400, Minibatch Loss: 0.034726, Training Accuracy: 0.992188
Iter 13500, Minibatch Loss: 0.044745, Training Accuracy: 0.984375
Iter 13600, Minibatch Loss: 0.091961, Training Accuracy: 0.968750
Iter 13700, Minibatch Loss: 0.018736, Training Accuracy: 1.000000
Iter 13800, Minibatch Loss: 0.094984, Training Accuracy: 0.976562
Iter 13900, Minibatch Loss: 0.042751, Training Accuracy: 0.992188
Iter 14000

Iter 24800, Minibatch Loss: 0.011746, Training Accuracy: 1.000000
Iter 24900, Minibatch Loss: 0.009025, Training Accuracy: 1.000000
Iter 25000, Minibatch Loss: 0.018634, Training Accuracy: 0.992188
========> Validation Accuarcy: 0.948400, Testing Accuarcy: 0.944300
Iter 25100, Minibatch Loss: 0.009597, Training Accuracy: 1.000000
Iter 25200, Minibatch Loss: 0.019223, Training Accuracy: 0.992188
Iter 25300, Minibatch Loss: 0.010086, Training Accuracy: 1.000000
Iter 25400, Minibatch Loss: 0.017891, Training Accuracy: 0.992188
Iter 25500, Minibatch Loss: 0.002962, Training Accuracy: 1.000000
Iter 25600, Minibatch Loss: 0.040060, Training Accuracy: 0.984375
Iter 25700, Minibatch Loss: 0.007816, Training Accuracy: 1.000000
Iter 25800, Minibatch Loss: 0.008849, Training Accuracy: 1.000000
Iter 25900, Minibatch Loss: 0.013440, Training Accuracy: 1.000000
Iter 26000, Minibatch Loss: 0.007615, Training Accuracy: 1.000000
Iter 26100, Minibatch Loss: 0.021868, Training Accuracy: 0.992188
Iter 262